In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import wandb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('./data/df_for_ML.csv', head)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.drop(['fireplace_0', 
         'fireplace_1', 
         'fireplace_other',
         'pool',
         'baths',
         'sqft',
         'price_per_sqft'
         ], axis=1, inplace=True)

In [ ]:
df.info()

In [4]:
df.head()

,sqft,beds,lat,lon,pool,baths,property_type_condo,property_type_land,property_type_multi_family,property_type_other,...,fireplace_1,fireplace_other,state,year_built,remodeled_year,price_per_sqft,avg_school_rating,schools_qty,avg_school_dist,target
0,2900,4,35.627887,-77.335173,0,3.5,0,0,0,0,...,0,1,37,2019,0,144.0,3.25,8,5.54,418000.0
1,1947,3,47.699482,-117.190979,0,3.0,0,0,0,0,...,0,0,53,2019,0,159.0,0.27,3,1.33,310000.0
2,3000,3,33.943243,-117.376310,1,2.0,0,0,0,0,...,1,0,6,1961,1967,965.0,0.67,3,1.96,2895000.0
3,6457,5,32.702323,-97.347071,0,8.0,0,0,0,0,...,1,0,48,2006,2006,371.0,0.92,4,0.75,2395000.0
4,0,0,27.041567,-82.136146,0,0.0,0,1,0,0,...,0,0,12,0,0,0.0,0.47,3,4.08,5000.0


In [3]:
X = df.drop('target', axis=1)
y = df['target']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Создание и обучение модели случайного леса для регрессии
model = RandomForestRegressor(n_estimators=100, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print(f'Среднеквадратичная ошибка: {mse}')






Среднеквадратичная ошибка: 1046054232207.8867


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import load_iris

y_pred = model.predict(X_test)

# Вычисление точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Вывод матрицы ошибок и отчета классификации
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Коэффициенты модели
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)


In [ ]:
# Прогнозируем метки для тестовой выборки
y_pred = model.predict(X_test)

accuracy = (y_pred == y_test).mean()
print(f"Точность: {accuracy}")

# # Оцениваем точность модели
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy * 100:.2f}%')

# # Получаем расширенный отчет по классификации
# print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Загрузка данных
df = pd.read_csv('./data/df_for_ML.csv')

# Предположим, что целевая переменная называется 'price'
X = df.drop('target', axis=1)
y = df['target']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Создание и обучение модели дерева решений
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Важность признаков
print('Feature Importances:', model.feature_importances_)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Важность признаков
print('Feature Importances:', model.feature_importances_)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Загрузка набора данных
iris = load_iris()
X, y = iris.data, iris.target

# Разделение набора данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Инициализация случайного леса
clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

# Обучение модели
clf.fit(X_train, y_train)

# Прогнозирование
y_pred = clf.predict(X_test)

# Оценка точности
accuracy = (y_pred == y_test).mean()
print(f"Точность: {accuracy}")
